<a href="https://colab.research.google.com/github/AnaadKaur/Forest-Fire-Detection/blob/main/Computing%20and%20Exporting%20Fire%20Regions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

## Run the two cells below for output wrap-around

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"lineWrapping": True}}})

## Importing libraries

In [3]:
import math
import altair as alt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pprint import pprint
import geemap
import matplotlib.pyplot as plt

## Authentication and Initialization

In [4]:
import ee
ee.Authenticate()

In [5]:
ee.Initialize(project='ee-anaad')

## Asset handling

In [6]:
#===============================================================================
# Move all assets from src_folder to dest_folder
#===============================================================================

def move_assets(src_folder, dest_folder):
    assets = ee.data.listAssets({'parent': src_folder})
    for asset in assets['assets']:
        reg = asset['id'].split('/')[-1]
        new_asset = dest_folder + '/' + asset['id'].split('/')[-1]
        ee.data.copyAsset(asset['id'], new_asset, True)
    # uncomment if you want to delete source asset
        # ee.data.deleteAsset(asset['id'])


#===============================================================================
# Delete all assets from src_folder
#===============================================================================

def delete_assets(src_folder):
    assets = ee.data.listAssets({'parent': src_folder})
    for asset in assets['assets']:
        ee.data.deleteAsset(asset['id'])

In [7]:
#===============================================================================
# Export a collection to drive or as asset
#===============================================================================

def export_to_drive(collection, output_filename):
    task = ee.batch.Export.table.toDrive(
      collection = collection,
      description = output_filename,
      fileFormat = 'csv',
    #   selectors = ['ID', 'date', 'area', 'geometry'],
      folder='anaad',
    )
    task.start()

def export_to_asset(collection, output_assetid, output_filename):
    export_task = ee.batch.Export.table.toAsset(
        collection = collection,
        description = output_filename,
        assetId = output_assetid,
    )
    export_task.start()

# Load district, block and microwatershed boundaries

In [8]:
district_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/India_district_boundaries')
aez_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/India_AEZ_boundaries')
block_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/AEZ_mapped_block_boundaries')

In [9]:
blocks_odisha = ee.FeatureCollection('projects/ee-anaad/assets/District_blocks/Odisha')
blocks_jharkhand = ee.FeatureCollection('projects/ee-anaad/assets/District_blocks/Jharkhand')
blocks_rajasthan = ee.FeatureCollection('projects/ee-anaad/assets/District_blocks/Rajasthan')
blocks_bihar = ee.FeatureCollection('projects/ee-anaad/assets/District_blocks/Bihar')

In [10]:
mws_angul = ee.FeatureCollection('projects/ee-anaad/assets/Microwatershed_boundaries/angul')
mws_masalia = ee.FeatureCollection('projects/ee-anaad/assets/Microwatershed_boundaries/masalia')
mws_pindwara = ee.FeatureCollection('projects/ee-anaad/assets/Microwatershed_boundaries/pindwara')
mws_mandalgarh = ee.FeatureCollection('projects/ee-anaad/assets/Microwatershed_boundaries/mandalgarh')
mws_mohanpur = ee.FeatureCollection('projects/ee-anaad/assets/Microwatershed_boundaries/mohanpur')

In [11]:
aoi_angul = ee.FeatureCollection('projects/ee-anaad/assets/Block_boundaries/angul')
aoi_masalia = ee.FeatureCollection('projects/ee-anaad/assets/Block_boundaries/masalia')
aoi_pindwara = ee.FeatureCollection('projects/ee-anaad/assets/Block_boundaries/pindwara')
aoi_mandalgarh = ee.FeatureCollection('projects/ee-anaad/assets/Block_boundaries/mandalgarh')
aoi_mohanpur = ee.FeatureCollection('projects/ee-anaad/assets/Block_boundaries/mohanpur')

In [12]:
roi_angul = None
roi_dumka = district_boundaries.filter(ee.Filter.eq('Name', 'Dumka'))
roi_sirohi = district_boundaries.filter(ee.Filter.eq('Name', 'Sirohi'))
roi_bhilwara = district_boundaries.filter(ee.Filter.eq('Name', 'Bhilwara'))
roi_gaya = district_boundaries.filter(ee.Filter.eq('Name', 'Gaya'))

In [13]:
block_names = {
     "angul",
    "masalia",
    "pindwara",
    "mandalgarh",
    "mohanpur"
}
district_from_block = {
    "angul": "angul",
    "masalia": "dumka",
    "pindwara": "sirohi",
    "mandalgarh": "bhilwara",
    "mohanpur": "gaya"
}

In [14]:
#===============================================================================
# Set *aoi*, *roi*, and *mws* from *block_name*
#===============================================================================

def set_block(block_name):
    if block_name=='angul':
        aoi = aoi_angul
        roi = roi_angul
        mws = mws_angul

    if block_name=='masalia':
        aoi = aoi_masalia
        roi = roi_dumka
        mws = mws_masalia

    if block_name=='pindwara':
        aoi = aoi_pindwara
        roi = roi_sirohi
        mws = mws_pindwara

    if block_name=='mandalgarh':
        aoi = aoi_mandalgarh
        roi = roi_bhilwara
        mws = mws_mandalgarh

    if block_name=='mohanpur':
        aoi = aoi_mohanpur
        roi = roi_gaya
        mws = mws_mohanpur

    return aoi, roi, mws

# Helper functions

In [15]:
#===============================================================================
# Set ID of fire_regions using uids of intersecting microwatersheds
#===============================================================================

def set_properties(fire_regions, mws):
    idList = ee.List.sequence(0,fire_regions.size().subtract(1))
    fire_regions_list = fire_regions.toList(fire_regions.size())

    def set_prop_helper(newSysIndex):
        feature = ee.Feature(fire_regions_list.get(newSysIndex))
        intersecting_mws = mws.filterBounds(feature.geometry())
        uidList = intersecting_mws.aggregate_array('uid').add(ee.Number(newSysIndex).toInt())
        uid = uidList.join('_')
        return feature.set('system:index', uid, 'ID', uid)#, 'geometry', feature.geometry())

    return ee.FeatureCollection(idList.map(set_prop_helper))

#===============================================================================
# Function to compute statistics for fire_regions
#===============================================================================

def compute_stats(fire_regions):
    num_fires = fire_regions.size().getInfo()
    min_area = fire_regions.aggregate_min('area').getInfo()
    max_area = fire_regions.aggregate_max('area').getInfo()
    mean_area = fire_regions.aggregate_mean('area').getInfo()
    total_area = fire_regions.aggregate_sum('area').getInfo()
    return num_fires, min_area, max_area, mean_area, total_area

In [26]:
#===============================================================================
# Function to get composite image with MODIS and VIIRS bands from landsat-8 image
#===============================================================================

def getNativeImage(image_lsat):
    path = image_lsat.get('system:index').getInfo()
    # projection = image_lsat.select('B7').projection().getInfo()

    image_mod09ga_id = "MODIS/061/MOD09GA/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:]
    try:
        ee.data.getAsset(image_mod09ga_id)
    except ee.EEException:
        print("MODIS/061/MOD09GA/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:], 'does not exist.')
        return None

    image_vnp09ga_id = "NOAA/VIIRS/001/VNP09GA/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:]
    try:
        ee.data.getAsset(image_vnp09ga_id)
    except ee.EEException:
        print("NOAA/VIIRS/001/VNP09GA/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:], 'does not exist.')
        return None

    image_mod09ga = ee.Image(image_mod09ga_id).select(['sur_refl_b01','sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07'])#.reproject(crs = projection['crs'], crsTransform = projection['transform'])
    image_mod09ga = image_mod09ga.clip(image_lsat.select('B7').geometry())#.updateMask(image_lsat.select('B7').mask())
    image_vnp09ga = ee.Image(image_vnp09ga_id).select(['I1','I2','I3','M1','M2','M3','M4','M5','M7','M8','M10','M11'])#.reproject(crs = projection['crs'], crsTransform = projection['transform'])
    image_vnp09ga = image_vnp09ga.clip(image_lsat.select('B7').geometry())#.updateMask(image_lsat.select('B7').mask())
    # image_mod11a1 = ee.Image("MODIS/061/MOD11A1/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:])
    # image_mod11a1 = image_mod11a1.select(['LST_Day_1km', 'LST_Night_1km', 'Emis_31', 'Emis_32']).reproject(crs = projection['crs'], crsTransform = projection['transform'])
    # image_mod11a1 = image_mod11a1.updateMask(manual_annotation_patch.mask()).clip(manual_annotation_patch.geometry())

    image = image_lsat.addBands(image_mod09ga).addBands(image_vnp09ga)
    return image


def getReprojectedImage(image_lsat):
    path = image_lsat.get('system:index').getInfo()
    projection = image_lsat.select('B7').projection().getInfo()

    image_mod09ga_id = "MODIS/061/MOD09GA/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:]
    try:
        ee.data.getAsset(image_mod09ga_id)
    except ee.EEException:
        print("MODIS/061/MOD09GA/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:], 'does not exist.')
        return None

    image_vnp09ga_id = "NOAA/VIIRS/001/VNP09GA/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:]
    try:
        ee.data.getAsset(image_vnp09ga_id)
    except ee.EEException:
        print("NOAA/VIIRS/001/VNP09GA/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:], 'does not exist.')
        return None

    image_mod09ga = ee.Image(image_mod09ga_id).select(['sur_refl_b01','sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07']).reproject(crs = projection['crs'], crsTransform = projection['transform'])
    image_mod09ga = image_mod09ga.clip(image_lsat.select('B7').geometry())#.updateMask(image_lsat.select('B7').mask())
    image_vnp09ga = ee.Image(image_vnp09ga_id).select(['I1','I2','I3','M1','M2','M3','M4','M5','M7','M8','M10','M11']).reproject(crs = projection['crs'], crsTransform = projection['transform'])
    image_vnp09ga = image_vnp09ga.clip(image_lsat.select('B7').geometry())#.updateMask(image_lsat.select('B7').mask())
    # image_mod11a1 = ee.Image("MODIS/061/MOD11A1/"+path[-8:-4]+'_'+path[-4:-2]+'_'+path[-2:])
    # image_mod11a1 = image_mod11a1.select(['LST_Day_1km', 'LST_Night_1km', 'Emis_31', 'Emis_32']).reproject(crs = projection['crs'], crsTransform = projection['transform'])
    # image_mod11a1 = image_mod11a1.updateMask(manual_annotation_patch.mask()).clip(manual_annotation_patch.geometry())

    image = image_lsat.addBands(image_mod09ga).addBands(image_vnp09ga)
    return image

# Sets of conditions to classify pixels as fire or non-fire

### [Schroeder](https://www.sciencedirect.com/science/article/pii/S0034425715301206)

In [16]:
#===============================================================================
# EQUATIONS (Schroeder)
#===============================================================================
# The following functions implement the equations in the paper:https://www.sciencedirect.com/science/article/pii/S0034425715301206.

def Seq1(image, r75, diff75):
    '''identifies potentially unambiguous active fire pixels
    builds on the ETM + active fire algorithm while accommodating small differences in OLI spectral channels'''
    return r75.gt(2.5).And(diff75.gt(0.3)).And(image.select('B7').gt(0.5))

#-------------------------------------------------------------------------------

def Seq2(image):
    '''highly energetic and extensive fires can lead to folding of the DN on channel 7
    thereby characterizing another condition of potentially unambiguous active fire pixels'''
    return image.select('B6').gt(0.8).And(image.select('B1').lt(0.2)).And(image.select('B5').gt(0.4).Or(image.select('B7').lt(0.1)))

#-------------------------------------------------------------------------------

def Seq3(r75, diff75):
    '''complementing the identification of unambiguous fire pixels, the thresholds in test (1) are relaxed
    and other candidate fire pixels are selected for further analyses'''
    return r75.gt(1.8).And(diff75.gt(0.17))

#-------------------------------------------------------------------------------

def Seq4and5(b7, r75, unamb_fires, water):
    '''all pixels satisfying test (3) must then meet the following set of fixed threshold and contextual tests
    in order to be classified as potential fire-affected pixels'''

    # means and standard deviations are computed ignoring pixels showing channel 7 reflectance <= zero, unambiguous fires, as well as water pixels.
    ignored_pixels = b7.lte(0).Or(unamb_fires.Or(water))
    kept_pixels = ignored_pixels.Not()
    r75 = r75.updateMask(kept_pixels)
    b7 = b7.updateMask(kept_pixels)

    reducers = ee.Reducer.mean().unweighted().combine(reducer2=ee.Reducer.stdDev().unweighted(),sharedInputs=True)
    stats_r75 = r75.reduceNeighborhood(reducers, ee.Kernel.square(30, 'pixels', False))
    stats_b7 = b7.reduceNeighborhood(reducers, ee.Kernel.square(30, 'pixels', False))

    potential_r75 = r75.gt(stats_r75.select('R75_mean').add(stats_r75.expression("(3*b('R75_stdDev') > 0.8) ? 3*b('R75_stdDev') : 0.8")))
    potential_b7 = b7.gt(stats_b7.select('B7_mean').add(stats_b7.expression("(3*b('B7_stdDev') > 0.08) ? 3*b('B7_stdDev') : 0.08")))
    return (potential_r75.And(potential_b7)).unmask(unamb_fires)

#-------------------------------------------------------------------------------

def Seq6(image):
    '''additional test for potential fires'''
    return (image.select('B7').divide(image.select('B6'))).gt(1.6)

#-------------------------------------------------------------------------------

def Seq7to9(image):
    '''water pixels are classified based on spectral profiling using reflectance data from all seven input channels
    two distinct tests are applied in order to map oceans and inland water bodies'''
    return image.select('B4').gt(image.select('B5'))\
    .And(image.select('B5').gt(image.select('B6')))\
    .And(image.select('B6').gt(image.select('B7')))\
    .And((image.select('B1').subtract(image.select('B7'))).lt(0.2))\
    .And(image.select('B3').gt(image.select('B2'))
        .Or(image.select('B1').gt(image.select('B2'))
            .And(image.select('B2').gt(image.select('B3')))
            .And(image.select('B3').gt(image.select('B4')))
        )
    )

#-------------------------------------------------------------------------------

def getFireMaskSchroeder(image):

    r75 = image.select('B7').divide(image.select('B5')) # Compute only once (used by multiple equations).
    r75 = r75.select(['B7'], ['R75'])
    diff75 = image.select('B7').subtract(image.select('B5')) # Compute only once (used by multiple equations).
    diff75 = diff75.rename(['D75'])

    # Unambiguous fires (satisfy eq 1 or 2).
    unamb_fires = Seq1(image, r75, diff75).Or(Seq2(image))

    # Potential fires (satisfy eq 3).
    potential_fires = Seq3(r75, diff75)

    # Test eq 6 before eq 4 and 5 in an attempt to avoid the time-consuming contextual test when possible.
    potential_fires = potential_fires.And(Seq6(image))

    # Water pixels (used by the contextual test and excluded from the result.
    water = Seq7to9(image)

    # Contextual test for potential fires (eq 4 and 5).
    potential_fires = potential_fires.And(Seq4and5(image.select('B7'), r75, unamb_fires, water))

    return (water.Not().And(unamb_fires.Or(potential_fires))).rename(['F'])

## [Murphy](https://www.sciencedirect.com/science/article/abs/pii/S0034425716300554)

In [17]:
#===============================================================================
# EQUATIONS (MURPHY)
#===============================================================================
# The following functions implement the equations in the paper:https://www.sciencedirect.com/science/article/abs/pii/S0034425716300554.

def Meq2(image):
    '''a pixel has an α of one if reflectance in band 7 is at least 40% greater than reflectance in both bands 5 and 6
    (i.e. because the 2.2 μm waveband is expected to be relatively sensitive to hot targets),
    and if reflectance in band 7 is not too low
    (this condition is imposed to avoid creating false alarms over low reflectivity surfaces such as water bodies).'''
    # Eq 2 (unambiguous fires)
    return (image.select('B7').divide(image.select('B6'))).gte(1.4)\
        .And((image.select('B7').divide(image.select('B5'))).gte(1.4))\
        .And(image.select('B7').gte(0.15))
#-------------------------------------------------------------------------------

def Meq3(image, unamb_fires, QA):
    '''a pixel will have a β of one if reflectance in band 6 is at least twice as high as in band 5,
    and if reflectance in band 6 is at least 0.5.
    the latter fixed threshold is to ensure that the pixel is sufficiently radiant;
    algorithm performance is not sensitive to this threshold.
    alternatively, a pixel could also be assigned a β of one if either bands 6 or 7 are saturated.'''

    # saturation
    def get_sat(fromBand, toBand):
        bitmask = ee.Number(1).leftShift(toBand-fromBand+1).subtract(1)
        return QA.rightShift(fromBand-1).bitwiseAnd(bitmask)

    # vals = [2724,2756,2804,2980,3012,3748,3780,6820,6852,6900,7076,7108,7844,7876,
    #         2728,2760,2808,2984,3016,3752,3784,6824,6856,6904,7080,7112,7848,7880,
    #         2732,2764,2812,2988,3020,3756,3788,6828,6860,6908,7084,7116,7852,7884]

    # sat = ee.Image(0).setDefaultProjection(image.select('B7').projection())
    # for val in vals:
    #     sat = sat.Or(QA.eq(val))

    # Eq 3 (potential fires)
    neighborhood = unamb_fires.reduceNeighborhood(ee.Reducer.sum(), ee.Kernel.square(1, 'pixels', False))
    return neighborhood.gt(0)\
        .And(
            ((image.select('B6').divide(image.select('B5'))).gte(2.0)
            .And(image.select('B6').gte(0.5)))
            .Or(get_sat(6,7))
            # .Or(sat)
        )

#-------------------------------------------------------------------------------

def getFireMaskMurphy(image):
    '''This is the central function. Receives the (corrected) reflectance bands
and a binary mask indicating saturated pixels, and returns a binary fire mask.'''
    QA = image.select('QA_RADSAT')
    image = image.select(['B5', 'B6', 'B7'])#.divide(math.sin(math.radians(image.get('SUN_ELEVATION').getInfo())))
    unamb_fires = Meq2(image)
    potential_fires = Meq3(image, unamb_fires, QA)
    return (unamb_fires.Or(potential_fires)).rename(['F'])

## Random Forests

In [18]:
def getFireMaskOldRandomForest(image):
    asset_id = 'projects/ee-anaad/assets/Classifiers/random_forest_classifier' # 75-25
    random_forest_classifier = ee.Classifier.load(asset_id)
    return image.classify(random_forest_classifier).rename(['F'])

In [19]:
def getFireMaskRFLandsat(image):
    asset_id = 'projects/ee-anaad/assets/Classifiers/RF_Landsat-8' # 100-0
    random_forest_classifier = ee.Classifier.load(asset_id)
    return image.classify(random_forest_classifier).rename(['F'])

In [20]:
def getFireMaskRFNativeProjections(image_lsat):
    asset_id = 'projects/ee-anaad/assets/Classifiers/RF_Landsat-8_MODIS_VIIRS_native_projections'
    random_forest_classifier = ee.Classifier.load(asset_id)

    image = getNativeImage(image_lsat)
    if image:
        return image.classify(random_forest_classifier).rename(['F'])
    else:
        return ee.Image(0).setDefaultProjection(image_lsat.select('B7').projection())

In [21]:
def getFireMaskRFReprojected(image_lsat):
    asset_id = 'projects/ee-anaad/assets/Classifiers/RF_Landsat-8_MODIS_VIIRS_reprojected_to_30m'
    random_forest_classifier = ee.Classifier.load(asset_id)

    image = getReprojectedImage(image_lsat)
    if image:
        return image.classify(random_forest_classifier).rename(['F'])
    else:
        return ee.Image(0).setDefaultProjection(image_lsat.select('B7').projection())

# Compute fire regions, get/plot statistics, and export as asset

In [22]:
#===============================================================================
# Helper Functions
#===============================================================================

# Function to compute fire regions for aoi from start_date to end_date using forest_mask
def compute_fire_regions(aoi, start_date, end_date, forest_mask, getFireMask):
    aoi_buffered = aoi.geometry().buffer(1000)
    imageCollection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
        .filterDate(start_date, end_date)\
        .filterBounds(aoi_buffered)
    n = imageCollection.size().getInfo()
    imageList = imageCollection.toList(n)

    fire_regions = ee.FeatureCollection([])
    for i in range(n):
        image = (ee.Image(imageList.get(i))).clip(aoi_buffered)
        fire_mask = getFireMask(image)#.updateMask(forest_mask)
        fire = fire_mask.updateMask(fire_mask)
        fire_vectors = fire.addBands(fire).reduceToVectors(
            geometry = aoi_buffered,
            scale = 30,
            reducer = ee.Reducer.mean(),
            maxPixels = 1e10
        )
        fire_vectors = fire_vectors.select([]).map(lambda feature: feature.set({'date': image.date().format('YYYY-MM-dd'),'area': feature.area(maxError=1)}))
        fire_regions = fire_regions.merge(fire_vectors)
        fire_regions = fire_regions.distinct(['date', 'area', '.geo'])

    return fire_regions

In [24]:
# Input block_name, start_date and end_date
block_name = 'masalia'
start_date = '2019-07-01'
end_date = '2020-07-01'

# Input method to generate fire regions
method = 'OldRandomForest'

In [ ]:
#===============================================================================
# Compute fire_regions and export as asset
#===============================================================================


if method == 'Murphy':
    getFireMask = getFireMaskMurphy
elif method=='Schroder':
    getFireMask = getFireMaskSchroeder

elif method=='OldRandomForest':
    getFireMask = getFireMaskOldRandomForest
elif method=='RFLandsat':
    getFireMask = getFireMaskRFLandsat
elif method=='RFNativeProjections':
    getFireMask = getFireMaskRFNativeProjections
elif method=='RFReprojected':
    getFireMask = getFireMaskRFReprojected


aoi, roi, mws = set_block(block_name)

print("\n\nComputing fire regions for", block_name, "using", method)

loopStart = datetime.strptime(start_date,"%Y-%m-%d")
loopEnd = datetime.strptime(end_date,"%Y-%m-%d")
while loopStart < loopEnd:
    curr_start_date = loopStart.strftime("%Y-%m-%d")
    curr_end_date = (loopStart+relativedelta(years=1)-timedelta(days=1)).strftime("%Y-%m-%d")

    lulc = ee.Image('projects/ee-anaad/assets/LULC_outputs/'+block_name+'_'+curr_start_date+'_'+curr_end_date+'_LULCmap_30m')
    forest_mask = lulc.eq(6)
# uncomment the line below to add forest masks to the Map initialized above
    # Map.addLayer(forest_mask, {'min':0, 'max':1, 'palette': ['black', 'white']}, 'forest_mask'+'_'+curr_start_date+'_'+curr_end_date)

    print("\nComputing fire regions from", curr_start_date, "to", curr_end_date,"\n")

    loopStart = loopStart+relativedelta(years=1)
    fire_regions = compute_fire_regions(aoi, curr_start_date, loopStart.strftime("%Y-%m-%d"), forest_mask, getFireMask)
    fire_regions = set_properties(fire_regions, mws)

    output_filename = block_name + '_' + curr_start_date + '_' + curr_end_date
    output_assetid = 'projects/ee-anaad/assets/'+method+'_fire_regions/'+output_filename+'test'
    export_to_asset(fire_regions, output_assetid, output_filename)